In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

In [5]:
def gaussian_lineshape(E, sig, shift, w):
  Fk = []
  for i in range(len(E)):
    num = -((w - E[i] + shift[i])**2)
    deno  = 2*((sig[i])**2)

    temp = np.exp((num)/(deno))
    Fk.append(temp)

  Fk = np.array(Fk)

  return Fk

In [11]:
def data_generation_constraints(M, W, E0, mu0, sig0, gaussian_shift, gaussian_amp):  

  ir_data = []
  for i in range(M): 
    # this adds noise to the Ek values
    delta = np.random.uniform(low=-20, high=20, size=(len(E0)))
    E = E0 - delta

    # this adds noise to the uk values
    delta = np.random.uniform(low=0, high=0.05, size=(len(mu0)))
    mu = mu0*(1 + delta)

    if(random.randint(0,1) == 0):
      shiftl = gaussian_shift*np.random.uniform(low=0, high=1, size=(len(E0)))
      shiftr = (gaussian_shift-shiftl)*np.random.uniform(low=0, high=1, size=(len(E0)))
    else:
      shiftr = gaussian_shift*np.random.uniform(low=0, high=1, size=(len(E0)))
      shiftl = (gaussian_shift-shiftr)*np.random.uniform(low=0, high=1, size=(len(E0)))

    #this controls amplitude of the subpeaks
    ampl = 1 + gaussian_amp*np.random.uniform(low=0, high=1, size=(len(E0)))
    ampr = 1 + gaussian_amp*np.random.uniform(low=0, high=1, size=(len(E0)))

    ir = []
    for j in range(len(W)):
      #subpeak on the centre
      Fkc = gaussian_lineshape(E, sig0, np.zeros(len(E0)), W[j])

      #subpeak on the left side, which depends on the subpeak on the center
      Fkl = ampl*gaussian_lineshape(E, sig0, -shiftl, W[j])

      #subpeak on the right side, which also depends on the subpeal on the center
      Fkr = ampr*gaussian_lineshape(E, sig0, shiftr, W[j])

      # #resultant peak(noisey)
      Fk = (Fkl + Fkc + Fkr)/3
      
      uk2 = mu**2

      #calculation of absorption spectra
      Iw_n = np.sum(uk2*Fk)

      ir.append(Iw_n) 
 
    ir_data.append(ir)

  return np.array(ir_data)

In [12]:
E0 = np.array([1723.9, 1495.4, 1229.2])
mu0 = np.array([9.0, 7.9, 4.6])

sig0 = np.array([15, 20, 25])

gaussian_shift = 2.25*sig0 #this values are determined by trial and error

Lambda = 0.03

In [13]:
E_n = np.array([[0.1130, 0.6488, 0.7859],
               [0.3010, 0.7178, 0.7567],
               [0.8149, 0.8018, 0.2423],
               [0.1755, 0.4216, 0.6776],
               [0.6333, 0.8440, 0.9435]])

mu_n = np.array([[0.4930, 0.9186, 0.0396],
                 [0.7459, 0.9850, 0.0857],
                 [0.1391, 0.2606, 0.2065],
                 [0.9943, 0.7683, 0.7363],
                 [0.4979, 0.4914, 0.2236]])

sig_n = np.array([[0.1897, 0.9807, 0.6175],
                  [0.1254, 0.2593, 0.8311],
                  [0.8478, 0.2729, 0.2619],
                  [0.6161, 0.9742, 0.0100],
                  [0.5866, 0.6545, 0.2928]])

In [ ]:
W = np.arange(1000, 2000, 2)

for i in range(5):
  E = E0*(1 + (Lambda*(E_n[i] - 0.5)))
  mu = mu0*(1 + (Lambda*(mu_n[i] - 0.5)))
  sig = sig0*(1 + (Lambda*(sig_n[i] - 0.5)))

  data = data_generation_constraints(1, W, E, mu, sig, gaussian_shift, 0.5, i)

  df = pd.DataFrame()

  for j in range(len(W)):
    df[f"{j}"] = data[:, j]

  df.to_csv(f"/content/drive/MyDrive/BTP/Data/final/final_ir/{i}.csv", header=True)